In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
#%% read spectrum for seperation on 1dr
full_spectra = np.loadtxt(open("f15 1800w 1.00beamdia 3v 3 layer z-7 (1Dr).SOMS", "rb"), delimiter="\t")
avg_spectra = np.mean(full_spectra, axis = 0)
plt.plot(avg_spectra)
plt.ylim(1000, 1200)

FileNotFoundError: [Errno 2] No such file or directory: 'f15 1800w 1.00beamdia 3v 3 layer z-7 (1Dr).SOMS'

In [ ]:
full_spectra.shape, avg_spectra.shape

((2039, 7377), (7377,))

In [ ]:
threshold = 1040
up_pos = np.zeros(3)
up_j = 0
down_pos = np.zeros(3)
down_j = 0

In [ ]:
for i in range(len(avg_spectra) - 1):
    if (avg_spectra[i] > threshold) & (avg_spectra[i - 1] < threshold):
        up_pos[up_j] = i
        up_j += 1
    if (avg_spectra[i] > threshold) & (avg_spectra[i + 1] < threshold):
        down_pos[down_j] = i
        down_j += 1

In [3]:
up_pos, down_pos

NameError: name 'up_pos' is not defined

In [7]:
# find the spectra length for active signals
len_spectra = int(min(down_pos - up_pos))
len_spectra

942

In [8]:
shift_pos = np.floor((down_pos - up_pos - np.ones(3) * len_spectra) / 2).astype(int)
start_pos = up_pos.astype(int) + shift_pos
end_pos = up_pos.astype(int) + shift_pos + np.ones(3).astype(int) * len_spectra

In [9]:
shift_pos

array([5, 0, 6])

In [10]:
# stack the spectra features of the three active layers
feature_spectra = np.loadtxt(open("extract_from_spectrum.csv", "rb"), delimiter=",")
feature_spectra_stack = np.zeros((len_spectra, feature_spectra.shape[1], 3))
for i in range(3):
    feature_spectra_stack[:, :, i] = feature_spectra[start_pos[i] : end_pos[i], :]

In [11]:
# read in the image features
feature_image = np.loadtxt(open("extract_from_images.csv", "rb"), delimiter=",")

In [12]:
#%% return the feature given any start and end location (0 - 35mm)
### 35mm 分配 spectrum 和 image
def feature_spectra_window(start_len, end_len):
    start_spectra = int(start_len / 35 * len_spectra)
    end_spectra = int(end_len / 35 * len_spectra)
    return(feature_spectra_stack[start_spectra : end_spectra, :, :])

def feature_image_window(start_len, end_len):
    start_spectra = int(start_len / 35 * feature_image.shape[0])
    end_spectra = int(end_len / 35 * feature_image.shape[0])
    return(feature_image[start_spectra : end_spectra, :])

In [24]:
# Test
spec = feature_spectra_window(0, 0.5)
print(spec.shape)
img = feature_image_window(0, 0.5)
print(img.shape)

(13, 16, 3)
(57, 6)
